In [28]:

import pandas as pd
import numpy as np
import pickle
import re

from sklearn.feature_extraction.text import CountVectorizer
from turftopic import KeyNMF

from datetime import datetime



/Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Load the data

In [3]:
scriptum_df = pd.read_csv('data cleaning/scriptum_cleaned_data.csv')

In [4]:
scriptum_df.drop(columns=['Unnamed: 0'], inplace=True)
scriptum_df.drop(columns=['ANNOTATION'], inplace=True)
print(scriptum_df.shape[0])
scriptum_df.head(2)

5642


,periodical_category,periodical_href,periodical_title,periodical_basic_info,file_url,file,year,pages_N,tokens_N,text,cleaned_text
0,samizdat,/cs/periodika/ps-psi,Ψ [Psí],"['', ' [Praha] ', ' 1985 – 1989 ', ' 1 × ročně...",https://files.scriptum.cz/scriptum/%5Bnode%5D/...,psi_1986_2_vina_ocr.txt,1986.0,36,24524,V V I N N V V I N N N A V V I N N N A A V V I ...,V V I N N V V I N N N A V V I N N N A A V V I ...
1,samizdat,/cs/periodika/ps-psi,Ψ [Psí],"['', ' [Praha] ', ' 1985 – 1989 ', ' 1 × ročně...",https://files.scriptum.cz/scriptum/%5Bnode%5D/...,psi_1987_3_freud-a-psychoanalyza_ocr.txt,1987.0,42,21575,[ 3 ] 0000000 0 ooooon 0 uo 0000000 0 0 0 o 0 ...,[ 3 ] 0000000 0 ooooon 0 uo 0000000 0 0 0 o 0 ...


In [5]:
scriptum_df[scriptum_df.file.str.contains('obsah_ocr')]

,periodical_category,periodical_href,periodical_title,periodical_basic_info,file_url,file,year,pages_N,tokens_N,text,cleaned_text


In [6]:
scriptum_df = scriptum_df[scriptum_df.year.notna()]
scriptum_df.shape[0]

5552

In [7]:
texts = list(scriptum_df['cleaned_text'])
texts[:3]

['V V I N N V V I N N N A V V I N N N A A V V I N N N A A V V I N N N A A A A V I N N A A [ 2 ] + + + + , + + + . + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + + ++ + + +  Články Pocity viny a - vina 3 M.Boss: Analýza "pocit viny" a cíl psychoterapie 5  Vina a zkušenost viny dnes (M. Odermatt) 13 Kde jsou hranice viny (/.Górres) 19 Recenze K. Jaspers: Otázka viny 22 A. Görres, K.Rahner: Das Böse 24 Excerpta a reprinty V.Svobodová: Člověk v mezních situacích 26 J.Gründel: Mravní přičítatelnost viny 28 M.Buber: Vina a pocit viny 29 Meditace R.Guardini: Odpouštění 33 P o e z i e V. Holan 36 V. Renč 36 O d. ČLÁNKY Pocity viny a - vina Pro to, co nenabylo spojitosti se svou základnou nebo ji ztratilo, má naše mudrosloví trefné označení: visí to ve vzduchu, stojí to na písku« Příkladem něčeho takového mže být pojem pocitu viny v té podobě, v jaké s ním zacházejí mnozí dnešní psychopat o logové. Srovnáme-li jej s podobným pojmem ‘\'pocitu pronásledování“, zjistíme rozdíl« Sn

## Preprocessing text (final)

In [8]:
def preprocess(doc):
    # Included in data\ cleaning.py
    doc = re.sub(r'[°\[\]\\\(\)\"\'\<\>\+\-\=_\^„]', '', doc)
    doc = re.sub(r'\s{2,}', '', doc)
    doc = doc.strip().lower()
    return doc


In [9]:
texts_clean = [preprocess(text) for text in texts]

## Encoder

In [95]:
# model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 370085.65it/s]


In [98]:
# outputs = model.encode(texts, max_length=8192, batch_size=12,
#                        return_dense=True, return_sparse=True, return_colbert_vecs=True)

Inference Embeddings: 100%|██████████| 4/4 [03:21<00:00, 50.35s/it]


In [123]:
# # save numpy array outputs to directory
# np.save('dense_vecs.npy', outputs['dense_vecs'], allow_pickle=True) 
# np.save('lexical_weights.npy', outputs['lexical_weights'], allow_pickle=True) 
# # np.save('colbert_vecs.npy', outputs['colbert_vecs'], allow_pickle=True) 
# # open the numpy array
# dense_embeddings = np.load('dense_vecs.npy', allow_pickle=True)
# lexical_weights = np.load('lexical_weights.npy', allow_pickle=True)
# # colbert_embeddings = np.load('colbert_vecs.npy', allow_pickle=True)


# # pickle model
# with open('model.pkl', 'wb') as f:
#     pickle.dump(model, f)
# # load the model
# with open('model.pkl', 'rb') as f:
#     model = pickle.load(f)

In [ ]:
# from sklearn.cluster import KMeans

# # Apply KMeans clustering as a simple topic model surrogate
# num_topics = 10
# kmeans = KMeans(n_clusters=num_topics, random_state=42)
# clusters = kmeans.fit_predict(dense_embeddings)

# # Inspect clusters (topics)
# for topic_id in range(num_topics):
#     print(f"Topic {topic_id}:")
#     for i, doc in enumerate(texts[:1000]):


## Topic modeling

In [10]:
with open('stop-words-czech2.txt', 'r') as f:
    stop_words = f.read().splitlines()

stop_words = [re.sub('\\ufeff', '', word) for word in stop_words]

# Word frequency and lemmatization ???

In [11]:
ts = [datetime(year=year_str, month=1, day=1) for year_str in scriptum_df['year'].astype(int)]

In [12]:
N = scriptum_df.shape[0]

In [13]:
n_topics = 200

### KeyNMF

**NOTE**: lemmatization should be done  
Maybe reduce amount of texts  
Maybe set hyperparameters like vocabulary  
**For feminism topic:**  
seed_phrase	Optional[str]	Describes an aspect of the corpus that the model should explore. It can be a free-text query, such as "Christian Denominations: Protestantism and Catholicism"	None

In [29]:
min_df, max_df = int(0.0007*N), int(0.95*N)
min_df, max_df, N

(3, 5274, 5552)

In [30]:
model_keynmf = KeyNMF(n_topics, top_n=40, encoder="paraphrase-multilingual-mpnet-base-v2", 
                      vectorizer=CountVectorizer(min_df=int(0.0001*N), 
                                                 max_df=int(0.99*N),
                                                 stop_words=stop_words),
                      random_state = 42)

In [31]:
document_topic_matrix = model_keynmf.fit_transform_dynamic(
    texts_clean[:N], timestamps=ts[:N], bins=20
)

# 100=N 20sec=time paraphrase-MiniLM-L3-v2
# 100=N 35sec=time paraphrase-multilingual-mpnet-base-v2

InvalidParameterError: The 'min_df' parameter of CountVectorizer must be a float in the range [0.0, 1.0] or an int in the range [1, inf). Got 0 instead.

In [282]:
model_keynmf.plot_topics_over_time()

In [225]:
# pickle model
with open('model_keynmf_morning.pkl', 'wb') as f:
    pickle.dump(model_keynmf, f)

In [285]:
model_keynmf.extract_keywords(texts_clean[10])

ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.

In [279]:
model_keynmf.print_topics()

In [20]:
# !pip install ipywidgets

In [91]:
# model_keynmf.fit(texts_clean[:N])

Output()

[01:03:02] Keyword extraction done.                                                                   ]8;id=441701;file:///Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/turftopic/models/keynmf.py\keynmf.py]8;;\:]8;id=412099;file:///Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/turftopic/models/keynmf.py#228\228]8;;\

           Model fitting done.                                                                        ]8;id=65832;file:///Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/turftopic/models/keynmf.py\keynmf.py]8;;\:]8;id=255877;file:///Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/turftopic/models/keynmf.py#235\235]8;;\

KeyNMF(encoder='paraphrase-MiniLM-L3-v2', n_components=20, top_n=20,
       vectorizer=CountVectorizer(min_df=10,
                                  stop_words=['a', 'aby', 'aj', 'ale', 'ani',
                                              'aniž', 'ano', 'asi', 'až', 'bez',
                                              'bude', 'budem', 'budeš', 'by',
                                              'byl', 'byla', 'byli', 'bylo',
                                              'být', 'co', 'což', 'cz', 'či',
                                              'článek', 'článku', 'články',
                                              'další', 'dnes', 'do', 'ho', ...]))

In [255]:
(document_topic_matrix == 0).sum() / (document_topic_matrix.shape[0]*document_topic_matrix.shape[1]) * 100

np.float64(84.32375720461096)

In [284]:
model_keynmf.print_representative_documents(0, texts_clean[:N], document_topic_matrix)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [260]:
model_keynmf.print_topic_distribution(texts_clean[0])

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Topic name                                                         ┃ Score ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ 191_svj_zas_jakou_komu                                             │  0.00 │
│ 172_vedou_traduje_básně_hleděl                                     │  0.00 │
│ 194_protisovětské_protistátní_protisocialistické_protikomunistické │  0.00 │
│ 180_právech_stíhaných_výslech_zúčastnil                            │  0.00 │
│ 185_svaté_božímu_zbožnosti_boží                                    │  0.00 │
│ 127_ježíšova_ježíše_ježíšovy_ježíšovu                              │  0.00 │
│ 117_byly_kterém_měla_měly                                          │  0.00 │
│ 78_diskuse_teoretickopolitický_občasník_diskusi                    │  0.00 │
│ 22_zahraničíczechoslovak_jubilejního_jubilejním_sletové            │  0.00 │
│ 190_vydal_vydalo_vydala_přednesl                                   │  0.00 │
└────────────────────────────────────────────────────────────────────┴───────┘

In [272]:
!pip uninstall openai -y
!pip install openai --upgrade

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: openai 1.82.0
Uninstalling openai-1.82.0:
  Successfully uninstalled openai-1.82.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached openai-1.82.0-py3-none-any.whl (720 kB)
You should consider upgrading via the '/Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/bin/python3 -m pip install --upgrade pip' command.


In [274]:
# !python -c "import openai; print(openai.__version__)"
!which python

/Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/bin/python


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [278]:
!pip list | grep -E 'openai|turftopic'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


openai                    1.82.0
turftopic                 0.17.0
You should consider upgrading via the '/Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/bin/python3 -m pip install --upgrade pip' command.


In [275]:
import sys
print(sys.executable)

/Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/bin/python


In [277]:
from turftopic.namers import OpenAITopicNamer

namer = OpenAITopicNamer("gpt-4o-mini")
model_keynmf.rename_topics(namer)
model_keynmf.print_topics()

ModuleNotFoundError: In order to use OpenAITopicNamer you'll need to install via;

pip install openai



In [261]:
!pip install topic-wizard

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     |████████████████████████████████| 112 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 500 kB 14.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 43.8 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 19.4 MB/s eta 0:00:01
     |████████████████████████████████| 7.8 MB 25.5 MB/s eta 0:00:01
     |████████████████████████████████| 227 kB 25.7 MB/s eta 0:00:01
     |████████████████████████████████| 101 kB 20.3 MB/s ta 0:00:01
     |████████████████████████████████| 114 kB 23.9 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 444 kB 37.0 MB/s eta 0:00:01
  Using cached more_itertools-10.7.0-py3-none-any.whl (65 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
  Using cached pydantic_core-2.33.2-cp39-cp39-macosx_11_0_arm64.whl (1.9 MB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
     |████████████████████████████████| 56 kB 14.6 MB/

In [264]:
import topicwizard

topicwizard.visualize(texts_clean[:N], model=model_keynmf)

Output()

Preprocessing


KeyboardInterrupt: 

### S3


In [152]:
from turftopic import SemanticSignalSeparation

model_s3 = SemanticSignalSeparation(n_components=20, 
                                    feature_importance="angular", 
                                    random_state=42,
                                    encoder='paraphrase-MiniLM-L3-v2',
                                    max_iter=1000,
                                    vectorizer=CountVectorizer(min_df=5, max_df=0.8, 
                                                               stop_words=stop_words))

In [153]:
output_s3 = model_s3.fit_dynamic(texts_clean[:N], 
                                timestamps=ts[:N], 
                                bins=10)

Output()

[01:39:48] Documents encoded.                                                                         ]8;id=663641;file:///Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=921277;file:///Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/turftopic/models/decomp.py#141\141]8;;\

/Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/sklearn/decomposition/_fastic
a.py:127: ConvergenceWarning:

FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.

[01:39:49] Decomposition done.                                                                        ]8;id=994985;file:///Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=505822;file:///Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/turftopic/models/decomp.py#144\144]8;;\

[01:39:50] Term extraction done.                                                                      ]8;id=407140;file:///Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=256115;file:///Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/turftopic/models/decomp.py#147\147]8;;\

[01:40:01] Vocabulary encoded.                                                                        ]8;id=687489;file:///Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=871763;file:///Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/turftopic/models/decomp.py#157\157]8;;\

           Model fitting done.                                                                        ]8;id=681025;file:///Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/turftopic/models/decomp.py\decomp.py]8;;\:]8;id=322367;file:///Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/lib/python3.9/site-packages/turftopic/models/decomp.py#170\170]8;;\

In [ ]:
# pickle model
with open('model_s3.pkl', 'wb') as f:
    pickle.dump(model_s3, f)

In [126]:
# !pip install plotly

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     |████████████████████████████████| 16.1 MB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 4.0 MB/s eta 0:00:01
You should consider upgrading via the '/Users/evaqw/Documents/UniMarburg/Hackathon/SCRIPTUM/venv/bin/python3 -m pip install --upgrade pip' command.


In [154]:
model_s3.print_topics()

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Topic ID ┃ Highest Ranking                                   ┃ Lowest Ranking                                   ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        0 │ l3, l4, rz, tmi, l7                               │ grafická, člunu, graficky, grafický,             │
│          │                                                   │ architekturou                                    │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        1 │ mechaniky, oklamat, beznaděje, matematika,        │ pozoruje, pospas, pozoru, potrubí, pospolu       │
│          │ mechanika                                         │                                                  │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        2 │ lavic, sedla, lavice, prostoře, hloubka           │ neúprosný, příbuznými, jednoduchosti, zpoždění,  │
│          │                                                   │ zřetelný                                         │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        3 │ mírových, katolíci, následovníci, dokazoval,      │ ep, přebujelé, věcech, přečteme, postuláty       │
│          │ prodloužení                                       │                                                  │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        4 │ alternativa, alternativy, alternativní, jednat,   │ lano, rto, vtipného, hospodářském, vti           │
│          │ nemalou                                           │                                                  │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        5 │ sebereflexe, prožité, prožít, prožitk, přeložit   │ výšin, vh, vys, výs, vyš                         │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        6 │ 716, 792, 726, 723, 711                           │ zájem, zaujímají, zahájením, míjejí, zastihuje   │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        7 │ berou, pláty, platy, brodského, plochy            │ vyzařovat, panovníka, požáru, panovník, nazareta │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        8 │ tahy, tav, taa, táž, táhla                        │ netuší, scénami, absolvovali, nepřísluší,        │
│          │                                                   │ popisující                                       │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│        9 │ general, generál, brána, book, bank               │ neumdlévající, určující, podněcuje, určuje,      │
│          │                                                   │ odmítali                                         │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│       10 │ ospravedlňovat, doplňování, osnova, doplňovaly,   │ právním, píšeš, pilného, protestantských,        │
│          │ doplňovat                                         │ pokračovatelé                                    │
├──────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│       11 │ prmyslová, ponejprv, majky, prešově, spoločnosti  │ absolutnímu, 3000, řidič, universitní,           │
│          │                                            

In [145]:
model_s3.print_representative_documents(topic_id=1, raw_documents=texts_clean[:N],)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Document                                                                                             ┃ Score ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ d i s k u s e č. 39květen 1986 /teoretickcpolitický občasník/tohoto čísla: milánská výzva a co je    │  2.52 │
│ kolem /ladislav hejdánek/ aalvmelej. w dnech 16.18*května se v miláně konala mezinárodní konference  │       │
│ deset let po helsinkách. mezinárodní kolektiv složený z aktivist nezávislé mírové iniciativy z       │       │
│ vídně, ...                                                                                           │       │
├──────────────────────────────────────────────────────────────────────────────────────────────────────┼───────┤
│ d i s k u s e č. 19leden 1985 /teoretickopolitický občasník/tohoto čísla: rozcestí pochyb i nadějí   │  2.15 │
│ /laureatus/ na obranu vydavatele a nejen jeho /rudolf zukal/ laureatus rozcestí pochyb i nadějí      │       │
│ procházíme nesnadným obdobím ve vnitřních i mezinárodních vztazích. osmdesátá léta se naplňují       │       │
│ politický ...                                                                                        │       │
├──────────────────────────────────────────────────────────────────────────────────────────────────────┼───────┤
│ ilietif mzfcmz&fj o siíiužefrřu s c i u & v r m& s & w í i i spoubčngsti 5. 5leden 1sb9 miata        │  2.01 │
│ úvodníku gvr. 3. dopás j&í5 5 5 2 rodplflovl via zz pvqúlzvxvrjt d o v o l e j voják 2v3 i dopi*     │       │
│ ifís pyiwcacp/ ív ssk 3 jjópis l&js nóiičcku ií£3 * 4 resolnař zo fcp .oni lidskýah p;r4v 193 5      │       │
│ troj37 ffies ...                                                                                     │       │
├──────────────────────────────────────────────────────────────────────────────────────────────────────┼───────┤
│ 2 1989 zpravodaj č5. skautů â skautek laszlo nagy skauting pluje proti proudu mladí lidé jbo u       │  1.59 │
│ vnímaví.šlechetní a spontánní.mají představivost a nadšení,jsou idealističtí a edvážmý.současně však │       │
│ stájí nedostatek sňalostí a zkušeností a jsou snadno ovlivnitelní. dnes poprvé tato přeopřilii       │       │
│ vychováva...                                                                                         │       │
├──────────────────────────────────────────────────────────────────────────────────────────────────────┼───────┤
│ čtení do krosny 2 / 89 čtení do krosny 2 obsah i slovo redakce milý čtenáři! milý otče kardinále 3   │  1.57 │
│ svatý otec o kněžství 3 svatý otec o manželství 3 povolání 4 obdivuji a velebím boží vedení 8je      │       │
│ šance 10 trápení kněze se snoubenci 14 bh si volá koho chce, kdy chce a odkud chce 15 doufat proti   │       │
│ vší na...                                                                                            │       │
├──────────────────────────────────────────────────────────────────────────────────────────────────────┼───────┤
│ ...                                                                                                  │       │
├──────────────────────────────────────────────────────────────────────────────────────────────────────┼───────┤
│ bull skot nezjívxslio míftov&o bleu l&.£ lúzclařlv* ¿ji ¿ixllxiiklmoi c.lkveten loó ; prohlášení     │ -1.59 │
│ ¿závislého mírového sclruitííii 2 dopiď č. 1 3 ohlasy a reakce 4 k prohlášení se připojili 5 dopis   │       │
│ čt 2 5 o ctkání se zástupci francouzské iírové iniciativy cod£.£ 6 zpráva o setkaní se zástupci čs.  │       │
│ míro...                                                                                              │       │
├──────────────────────────────────────────────────────────────────────────────────────────────────────┼───────┤
│ diskuse č. 49leden 1987 /teoretickopolitick

In [146]:
model_s3.plot_topics_over_time()

In [127]:
# plotly import
# import plotly.graph_objects as gox
# import plotly.express as px
# import plotly.io as pio

model_s3.plot_topics_over_time()
